ノーマルのニューラルネットワーク

In [ ]:
# 分かち書きを行うためにjanomeというPythonパッケージをインストール
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.2 MB/s 


In [ ]:
import pandas as pd # CSVを含む表形式データを読み込み，分析するためのパッケージ

# CSVファイルを読み込み
dataset = pd.read_csv('drive/MyDrive/実習B/slice_AI.csv')
dataset

,id,label,text
0,1,1,One day I noticed. I gatta go to that stupid s...
1,2,1,Dear friend But.I can't even smile Like those ...
2,3,1,That's true.
3,4,1,何回歌ったっていいが大抵伝わらず
4,5,1,I’m irritated all the time.
...,...,...,...
27995,27996,0,(AK-69)Homie 聴きいとくてえーか? Where you from?
27996,27997,0,K-TOWNが産うんだ こいつぁDOPE FLOWGamble みてぇな Scramble交...
27997,27998,0,あいつも Struggleでもココ迷子まいご 浴あびねぇライト
27998,27999,0,で 夢破ゆめやぶれてまうなんて 奴やつダサいぞ俺おれの解答かいとう ほりゃくたばるまで続つづ...


In [ ]:
from janome.tokenizer import Tokenizer # janomeからTokenizerクラスをimportする

# janomeはPythonで書かれた形態素解析器であり単語の分かち書きなどが可能
# なお，速度に難があるため，大量のテキストを処理するときには，MeCabを利用すること
tokenizer = Tokenizer(wakati=True)
# text列の文字列を単語に分割し保存する
dataset['text'] = dataset['text'].map(lambda x: list(tokenizer.tokenize(x)))
dataset

,id,label,text
0,1,1,"[One, , day, , I, , noticed, ., , I, , ga..."
1,2,1,"[Dear, , friend, , But, ., I, , can, ', t, ..."
2,3,1,"[That, ', s, , true, .]"
3,4,1,"[何, 回, 歌っ, たって, いい, が, 大抵, 伝わら, ず]"
4,5,1,"[I, ’, m, , irritated, , all, , the, , tim..."
...,...,...,...
27995,27996,0,"[(, AK, -, 69, ), Homie, , 聴き, い, とく, て, えー, ..."
27996,27997,0,"[K, -, TOWN, が, 産, うん, だ, , こいつ, ぁ, DOPE, , ..."
27997,27998,0,"[あいつ, も, , Struggle, で, も, ココ, 迷子, まいご, , 浴,..."
27998,27999,0,"[で, , 夢, 破, ゆめ, やぶれ, て, まう, なんて, , 奴, やつ, ダサ..."


In [ ]:
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T

# 単語辞書作成
text_vocab = build_vocab_from_iterator(dataset['text'], specials=('<unk>', '<pad>'))

# テキストの変換方法を定義
text_transform = T.Sequential(
    T.VocabTransform(text_vocab), # 単語を整数値に置き換え
    T.ToTensor(padding_value=text_vocab['<pad>']) # <pad>によって長さを揃える
)

stoi = text_vocab.get_stoi() # 各単語に割り当てられた整数値を取得（str to int）

In [ ]:
import torch
from torch.utils.data import DataLoader

train_size = 25000 # 先頭の20,000件を訓練データとして用いることにする
batch_size = 35 # ミニバッチのサイズを設定

def collate_batch(batch):
    # batchには，CSVの複数行（batch_sizeで指定している）がリストとして渡される．各行は[id, label, text]という形式である．
    texts = text_transform([text for (id, label, text) in batch]) # 定義したテキスト変換を適用
    labels = torch.Tensor([label for (id, label, text) in batch]) # ラベルは整数値なのでそのまま利用する．ただし，Tensor型というPyTorchで扱える型にしておく．ラベルが文字列などの場合は整数値に置き換える．
    return texts, labels

# 読み込みんだCSVのうち，先頭の `train_size` 件を訓練データ，残りをテストデータとして用いる
train_dataset, test_dataset = dataset[:train_size], dataset[train_size:]

# 訓練データ，および，テストデータ用のデータローダを用意する
train_data_loader = DataLoader(train_dataset.values, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_data_loader = DataLoader(test_dataset.values, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [ ]:
for i, (texts, labels) in enumerate(train_data_loader):
    print(i)
    for text, label in zip(texts, labels):
        print(text, label)
    break # 今回の例では1つのミニバッチを取り出したら強制的にループを抜けている

0
tensor([   82,   348,    18,    59,    48,  2893, 15386,    43,   359,  6616,
          154,   191,   669,  1546,   128,    51,    11,    10,    56,    77,
         6956,  7633,    12,    73,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1]) tensor(0.)
tensor([ 1862,     2,  1329,    50,     2,  2250,   949, 24750,  2677,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1]) tensor(1.)
tensor([17395,  1259,    19,   284, 11209,     9,    16,     2,  5072,  6318,
         4834,  3170,  3220,   263,    15,     3,  1254,    11,   145,     2,
         6244, 35565,   109,     5,    26,    94,  2102,     1,     1,     1,
  

In [ ]:
# ここではfastTextという単語埋め込みを利用する
from torchtext.vocab import FastText
fasttext = FastText(language="ja")

# fastTextも「辞書」を持っており，例えば，「の」という単語には整数の2が対応しており，
# さらに300次元の単語埋め込みが既に用意されている．
print(fasttext.itos[2], 2, fasttext.vectors[0])

.vector_cache/wiki.ja.vec: 1.37GB [01:37, 14.1MB/s]                            
100%|██████████| 580000/580000 [00:44<00:00, 12925.89it/s]


の 2 tensor([  2.4405,  -2.4237,  -2.3451, -12.3030,  -0.8740,  -1.8989,   2.7567,
          1.6199,   5.1494,  -4.1123,   0.3489,   4.1822,  -2.4252,  -3.2331,
          3.3351,  -1.1762,  -7.0971,  -3.4565,  -4.3095,   3.2371,   2.3855,
         -4.2671,  -2.7152,  -1.1240,   2.2580,  -6.9810,   4.3527,   2.1409,
          2.7580,  -6.1698,  -0.2339,  -1.7493,  -6.9762,  -8.5031,  -5.2302,
         -3.9934,  -4.2426,  -8.5977,   7.3990,  -4.9724,   2.7592,   4.1365,
          2.8621,  -4.2423,  -4.2332,  -2.1560,   1.9182,   3.3009,   2.8669,
          2.0220,  -5.0652,   3.4419,  -0.9964,   7.1275,   2.0687,  -1.2856,
         -1.4175,   4.1298,  -4.2781,   3.2888,   0.4101,   2.5506,  -0.1357,
          4.2976,  -3.6452,  -1.2767,   4.8675,   4.6114,   9.9612,  -6.9417,
          3.6101,   4.2744,  -6.4286,   6.0007,  -6.1865,   0.4312,   0.8587,
          3.3407,  -4.7399,   5.2652,  -2.9183,   7.4144,   2.6888,  17.5140,
          0.7445,   0.7603,   3.0601,   5.2463,  -2.9451,  -

In [ ]:
import torch
from torchtext.vocab import vocab

# 先程までは，自前の「辞書」を作成していたが，fastTextの辞書に切り替えるためにいくつか前処理を行う
# fastTextの辞書にない単語や<pad>などには，何も意味を持たないようなすべてが0であるベクトルを割り当てるようにする．
# このため，辞書のデフォルトインデックスを辞書の(一番最後の整数値+1)を割り当て，これがゼロベクトルになるようにする．
fasttext_vocab = vocab(fasttext.stoi)
special_token_id = len(fasttext)
fasttext_vocab.set_default_index(special_token_id)
shape = fasttext.vectors.shape
fasttext_vectors = torch.zeros((shape[0]+1, shape[1]))
fasttext_vectors[:shape[0]] = fasttext.vectors

# テキストの変換方法を再定義
text_transform = T.Sequential(
    T.VocabTransform(fasttext_vocab), # 単語をfastTextの辞書で置き換え
    T.ToTensor(padding_value=special_token_id) # (一番最後の整数値+1)という整数値で長さを揃える
)

# text_transformが変更されているため再定義
def collate_batch(batch):
    texts = text_transform([text for (id, label, text) in batch])
    labels = torch.Tensor([label for (id, label, text) in batch])
    return texts, labels

# 訓練データ，および，テストデータ用のデータローダを再度用意する
train_data_loader = DataLoader(train_dataset.values, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_data_loader = DataLoader(test_dataset.values, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

# 各単語に整数値が割り当てられ，また，整数値ごとに埋め込みが得られていることを確認
for i, (texts, labels) in enumerate(train_data_loader):
    for text, label in zip(texts, labels):
        print(text, label)
        for t in text[:10]: # 最初の10単語分だけ埋め込みを表示
          print(fasttext_vectors[t][:5]) # 300次元中の最初の5次元の値だけ表示
        break
    break # 1テキストだけで終了

tensor([580000,     13,    311, 580000,   6129, 580000,  17074, 580000,  92359,
        580000,  42458,   5248,   9899,   5578,     66,     91,     33,      1,
         12977, 580000, 580000,    138,    206,    118, 580000, 580000, 580000,
        580000, 580000, 580000]) tensor(0.)
tensor([0., 0., 0., 0., 0.])
tensor([  2.3096,  -2.2784,  -2.4345, -12.3790,  -0.8321])
tensor([ 1.5098, -1.6775, -1.5861, -7.9322, -0.6138])
tensor([0., 0., 0., 0., 0.])
tensor([ 1.5562, -1.3457, -1.5787, -7.9114, -0.7408])
tensor([0., 0., 0., 0., 0.])
tensor([ 1.7000, -1.3073, -1.9134, -7.9768, -0.3915])
tensor([0., 0., 0., 0., 0.])
tensor([ 1.1785, -0.7052, -0.9308, -4.5944, -0.3542])
tensor([0., 0., 0., 0., 0.])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextClassifier(nn.Module):
    def __init__(self):
        super(TextClassifier, self).__init__()

        # 単語埋め込み（FastText）を設定する
        self.embedding = torch.nn.Embedding.from_pretrained(
            embeddings=fasttext_vectors, freeze=False)
        
        emb_size = fasttext_vectors.shape[1]
        self.fc1 = nn.Linear(emb_size, 100) # 100 x 300 の行列を含む全結合層を設定
        self.fc2 = nn.Linear(100, 1) # 1 x 100 の行列（この場合はベクトル）を含む全結合層を設定

    def forward(self, seq):
        x = self.embedding(seq) # 各単語を単語埋め込みに変換する
        x = x.mean(axis=1) # 各テキストに含まれるすべての単語の埋め込みの平均をとる
        x = F.relu(self.fc1(x))
        h = self.fc2(x)
        h = h.squeeze()
        return h

In [ ]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # CPUもしくはGPUのどちらを使うかを設定
model = TextClassifier() # ニューラルネットワークモデルのインスタンスを生成
model = model.to(device) # CPUもしくはGPUのどちらを設定
optimizer = optim.Adam(model.parameters()) # 基本的な学習方法はミニバッチ勾配降下法ではあるが，その中でもよく用いられるAdamと呼ばれる方法を用いることにする

criterion = nn.BCEWithLogitsLoss() # 二値分類用の交差エントロピーを最小化することにする

epoch_size = 10 # 勾配降下法はすべてのデータでパラメータを更新したら終わりではなく，全データでの更新（=1エポック）を複数回行う必要がある

model.train() # モデルを学習モードに変更

# `epoch_size`の数だけ以下を繰り返す
for epoch in range(epoch_size):
    losses = []
    # イテレータはミニバッチ勾配降下法のために，`batch_size`で指定した数ごとにデータをわけて読み込んでくれる．
    for batch_idx, (texts, labels) in enumerate(train_data_loader):
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad() # 勾配の初期化
        y = model(texts) # 現時点でのモデルの出力を得る
        loss = criterion(y, labels.type(torch.float)) # 交差エントロピーの計算
        loss.backward() # 交差エントロピーの勾配計算
        optimizer.step() # パラメータ更新
        losses.append(loss.item())

    # 現在の交差エントロピーを出力
    print('Epoch: {}\tCross Entropy: {:.6f}'.format(epoch, sum(losses)))


Epoch: 0	Cross Entropy: 287.249658
Epoch: 1	Cross Entropy: 182.481160
Epoch: 2	Cross Entropy: 122.176008
Epoch: 3	Cross Entropy: 94.082269
Epoch: 4	Cross Entropy: 81.544879
Epoch: 5	Cross Entropy: 67.424019
Epoch: 6	Cross Entropy: 61.570901
Epoch: 7	Cross Entropy: 55.973914
Epoch: 8	Cross Entropy: 48.345930
Epoch: 9	Cross Entropy: 45.566885


In [ ]:
correct = 0
model.eval() # モデルを評価モードに変更
for batch_idx, (texts, labels) in enumerate(test_data_loader):
    texts, labels = texts.to(device), labels.to(device)
    y = model(texts) # モデルの出力を得る
    result = torch.sigmoid(y) # `TextClassifier`ではsigmoid関数を適用していなかったのでここで適用
    prediction = result >= 0.5 # `result`ベクトルと同じ次元を持ち，`result`の中で0.5以上である次元がTrue，それ以外がFalseであるベクトルを`prediction`とする
    target = labels == 1 # `labels`ベクトルと同じ次元を持ち，`labels`の中で1である次元がTrue，それ以外がFalseであるベクトルを`target`とする
    correct_num = target.eq(prediction).sum().item() # `prediction`ベクトルと`target`ベクトルでTrue/Falseが一致したものの数を数える
    correct += correct_num

# test_iterator.datasetにはテストデータ全体が入っているので，これの長さはテストデータの事例数となる
print("Accuracy: {:.3f}".format(correct / len(test_data_loader.dataset)))

Accuracy: 0.880
